# Priprema podataka

**Ucitavanje podataka iz datoteke**
  - u prvoj liniji su reci za koje treba naci regex (skup M)
  - u drugoj liniji su reci za koje ne treba naci regex (skup U)
  - reci u razdvojene sa: ", "

In [1]:
def readFile(filename):
    with open(filename, 'r') as f:
        match = [word for word in f.readline().split(", ")]
        # uklanjanje novog reda iz poslednje reci
        match[-1] = match[-1][:-1]
        unmatch = [word for word in f.readline().split(", ")]
        
    return match, unmatch

In [2]:
match, unmatch = readFile("example_1.txt")
print("Prvi skup: ", match)
print("Drugi skup: ", unmatch)

Prvi skup:  ['afoot', 'catfoot', 'dogfoot', 'fanfoot', 'foody', 'foolery', 'foolish', 'fooster', 'footage', 'foothot', 'footle', 'footpad', 'footway', 'hotfoot', 'jawfoot', 'mafoo', 'nonfood', 'padfoot', 'prefool', 'sfoot', 'unfool']
Drugi skup:  ['Atlas', 'Aymoro', 'Iberic', 'Mahran', 'Ormazd', 'Silipan', 'altared', 'chandoo', 'crenel', 'crooked', 'fardo', 'folksy', 'forest', 'hebamic', 'idgah', 'manlike', 'marly', 'palazzi', 'sixfold', 'tarrock', 'unfold']


In [3]:
# Broj reci u skupovima
num_m = len(match)
num_u = len(unmatch)
print(num_m)
print(num_u)

21
21


**Karakteri koji se pojavljuju u jednom skupu reci**

In [4]:
def charsInSet(wordSet):
    chars = []

    for word in wordSet:
        for c in word:
            if c not in chars:
                chars.append(c)

    chars.sort()

    return chars

In [5]:
chars_in_M = charsInSet(match)
print(chars_in_M)

['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'w', 'y']


**Opsezi reci (partial ranges) koji se javljaju u skupu**

In [6]:
# TODO - za skup match

**n-grami**

In [7]:
def ngram(M, U):
    res = {}

    for n in range(2, 5):
        for i in range(0, len(M)):
            word_m = M[i]
            word_u = U[i]
            
            ngrams_m = zip(*[word_m[i:] for i in range(n)])
            ngrams_u = zip(*[word_u[i:] for i in range(n)])

            gram_m = ["".join(gr) for gr in ngrams_m]
            gram_u = ["".join(gr) for gr in ngrams_u]

            for g in gram_m: # povecavamo score ako je u M
                if g not in res:
                    res[g] = 1
                elif g in res:
                    res[g] += 1

            for g in gram_u: # smanjujemo score ako je u U
                if g not in res:
                    res[g] = -1
                elif g in res:
                    res[g] -= 1

    return res

In [8]:
ngrams = ngram(match, unmatch)
ngrams = sorted(ngrams.items(), key=lambda x: x[1], reverse=True)
print(ngrams)

[('foo', 21), ('oo', 19), ('fo', 17), ('ot', 15), ('oot', 13), ('foot', 13), ('ool', 4), ('fool', 4), ('nfoo', 3), ('af', 2), ('tf', 2), ('nf', 2), ('od', 2), ('le', 2), ('ho', 2), ('ad', 2), ('afo', 2), ('tfo', 2), ('nfo', 2), ('ood', 2), ('hot', 2), ('pad', 2), ('afoo', 2), ('tfoo', 2), ('food', 2), ('ca', 1), ('at', 1), ('og', 1), ('gf', 1), ('er', 1), ('dy', 1), ('ol', 1), ('ry', 1), ('is', 1), ('sh', 1), ('os', 1), ('te', 1), ('ag', 1), ('ge', 1), ('th', 1), ('tp', 1), ('tw', 1), ('wa', 1), ('ay', 1), ('ja', 1), ('aw', 1), ('wf', 1), ('no', 1), ('on', 1), ('df', 1), ('pr', 1), ('ef', 1), ('sf', 1), ('cat', 1), ('atf', 1), ('dog', 1), ('ogf', 1), ('gfo', 1), ('fan', 1), ('anf', 1), ('ody', 1), ('ole', 1), ('ler', 1), ('ery', 1), ('oli', 1), ('lis', 1), ('ish', 1), ('oos', 1), ('ost', 1), ('ste', 1), ('ter', 1), ('ota', 1), ('tag', 1), ('age', 1), ('oth', 1), ('tho', 1), ('otl', 1), ('tle', 1), ('otp', 1), ('tpa', 1), ('otw', 1), ('twa', 1), ('way', 1), ('otf', 1), ('jaw', 1), ('awf

**Terminal i Function skupovi**

In [9]:
# . je placeholder za dete cvor
FUNCTION_SET = ['.*+', '.++', '.?+', '.{.,.}+', # possessive quantifiers
                '(.)',                          # group
                '[.]',                          # character class
                '[^.]',                         # negated character
                '..',                           # concatenator
                '.|.',                          # disjunction
                ]

In [10]:
TERMINAL_SET = ['a-z', 'A-Z', '0-9', '^', '$', '..', # instance independent terminals
                '\w', '\W', '\d', '\D', '\b', '\B', '\A', '\Z', '\s', '\S',
                chars_in_M,                          # instance dependent terminals
                ngrams
                ]